In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.fftpack import fft#푸리에 변환을 위한 코드입니다.
from scipy.stats import boxcox#박스콕스 변환을 위한 코드임
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
# ===== LightGBM 머신러닝 파이프라인 =====
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.express as px
import optuna
from optuna.samplers import TPESampler
from sklearn.linear_model import Ridge
from statsmodels.tsa.seasonal import STL
from sklearn.svm import SVR
#기타
import warnings
warnings.filterwarnings('ignore')

In [9]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVR
from tqdm import tqdm
from statsmodels.tsa.seasonal import STL

# 1. 데이터 불러오기 및 기본 전처리
df_train = pd.read_csv("train_heat.csv")
df_test = pd.read_csv("test_heat.csv")

df_train.columns = df_train.columns.str.replace('train_heat.', '', regex=False)
df_train = df_train.drop(columns=["Unnamed: 0"], errors='ignore')

df_test.columns = [
    "tm", "branch_id", "ta", "wd", "ws",
    "rn_day", "rn_hr1", "hm", "si", "ta_chi", "heat_demand"
]

# tm을 datetime으로 변환
df_train['tm'] = pd.to_datetime(df_train['tm'], format='%Y%m%d%H', errors='coerce')
df_test['tm'] = pd.to_datetime(df_test['tm'], format='%Y%m%d%H', errors='coerce')

# branch_id, tm 조합 유일성 체크
assert not df_train.duplicated(subset=['branch_id', 'tm']).any(), "df_train: branch_id, tm 중복 있음"
assert not df_test.duplicated(subset=['branch_id', 'tm']).any(), "df_test: branch_id, tm 중복 있음"

# 2. STL 시계열 분해 함수 (heat_demand는 제외)
def create_time_series_features(df, target_cols=['ta', 'ws'], freq_hours=23):
    df_features = df.copy()
    for branch in sorted(df['branch_id'].unique()):
        branch_mask = df_features['branch_id'] == branch
        branch_data = df_features.loc[branch_mask].copy().sort_values('tm')
        if len(branch_data) < freq_hours * 7:
            continue
        for col in target_cols:
            if col not in branch_data.columns:
                continue
            try:
                col_data = branch_data[col].interpolate().fillna(method='bfill').fillna(method='ffill')
                ts_data = col_data.copy()
                ts_data.index = pd.to_datetime(branch_data['tm'])
                stl = STL(ts_data, seasonal=freq_hours, robust=True)
                stl_result = stl.fit()
                indices = branch_data.index
                df_features.loc[indices, f'{col}_stl_trend'] = stl_result.trend.values
                df_features.loc[indices, f'{col}_stl_seasonal'] = stl_result.seasonal.values
                df_features.loc[indices, f'{col}_stl_resid'] = stl_result.resid.values
                df_features.loc[indices, f'{col}_detrend'] = col_data.values - stl_result.trend.values
                df_features.loc[indices, f'{col}_seasonal_strength'] = np.abs(stl_result.seasonal.values)
                seasonal_std = np.std(stl_result.seasonal.values)
                df_features.loc[indices, f'{col}_seasonal_volatility'] = seasonal_std
            except Exception as e:
                continue
    return df_features

# 3. 날씨 데이터 전처리 함수
def preprocess_weather_data(df):
    df = df.copy()
    # tm이 이미 datetime이면 변환 생략
    if not np.issubdtype(df['tm'].dtype, np.datetime64):
        df['tm'] = pd.to_datetime(df['tm'], format='%Y%m%d%H', errors='coerce')

    mask_outside_8_to_18 = (~df['tm'].dt.hour.between(8, 18)) & (df['si'] == -99)
    df.loc[mask_outside_8_to_18, 'si'] = 0
    df['wd'] = df['wd'].replace(9.9, np.nan)
    df.replace(-99, np.nan, inplace=True)
    df['heat_demand'] = (
    df.groupby('branch_id')['heat_demand']
      .transform(lambda x: x.interpolate(method='linear').fillna(method='ffill').fillna(method='bfill'))
)
    numeric_cols = ['ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si', 'ta_chi']

    for branch in df['branch_id'].unique():
        branch_mask = df['branch_id'] == branch
        branch_data = df.loc[branch_mask].copy()
        branch_data['hour'] = branch_data['tm'].dt.hour
        branch_data['day_of_year'] = branch_data['tm'].dt.dayofyear
        branch_data['month'] = branch_data['tm'].dt.month

        for col in numeric_cols:
            if col not in branch_data.columns:
                continue
            missing_mask = branch_data[col].isna()
            if missing_mask.sum() > 0:
                train_mask = ~missing_mask
                X_train = branch_data.loc[train_mask, ['hour', 'day_of_year', 'month']].values
                y_train = branch_data.loc[train_mask, col].values
                X_pred = branch_data.loc[missing_mask, ['hour', 'day_of_year', 'month']].values
                try:
                    scaler_X = StandardScaler()
                    scaler_y = StandardScaler()
                    X_train_scaled = scaler_X.fit_transform(X_train)
                    y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
                    svr = SVR(kernel='rbf', C=1.0, gamma='scale')
                    svr.fit(X_train_scaled, y_train_scaled)
                    X_pred_scaled = scaler_X.transform(X_pred)
                    y_pred_scaled = svr.predict(X_pred_scaled)
                    y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
                    df.loc[branch_mask & missing_mask, col] = y_pred
                except Exception as e:
                    df.loc[branch_mask, col] = df.loc[branch_mask, col].interpolate(method='linear')
            df.loc[branch_mask, col] = df.loc[branch_mask, col].fillna(method='ffill').fillna(method='bfill')

    df['year'] = df['tm'].dt.year
    df['month'] = df['tm'].dt.month
    df['day'] = df['tm'].dt.day
    df['hour'] = df['tm'].dt.hour
    df['weekday'] = df['tm'].dt.weekday
    df['is_weekend'] = df['weekday'].isin([5,6]).astype(int)

    df['peak_time1'] = 0
    df.loc[(df['hour'] >= 0) & (df['hour'] <= 6), 'peak_time1'] = 1
    df.loc[(df['hour'] > 6) & (df['hour'] <= 11), 'peak_time1'] = 2
    df.loc[(df['hour'] > 11) & (df['hour'] <= 18), 'peak_time1'] = 3
    df.loc[(df['hour'] > 18) & (df['hour'] <= 23), 'peak_time1'] = 4

    df['peak_time2'] = 0
    df.loc[(df['hour'] >= 2) & (df['hour'] <= 10), 'peak_time2'] = 1

    df['heating_season'] = df['month'].isin([10,11,12,1,2,3,4]).astype(int)

    df['temp_category20'] = pd.cut(df['ta'], bins=[-np.inf, 20, np.inf], labels=['low', 'high'])
    df['temp_category18'] = pd.cut(df['ta'], bins=[-np.inf, 18, np.inf], labels=['low', 'high'])
    df['temp_category16'] = pd.cut(df['ta'], bins=[-np.inf, 16, np.inf], labels=['low', 'high'])

    df = create_time_series_features(df, target_cols=['ta', 'ws'])

    df = df.sort_values(['branch_id', 'tm']).reset_index(drop=True)

    df['ta_mean_3h'] = df.groupby('branch_id')['ta'].transform(lambda x: x.rolling(3, min_periods=1).mean()).fillna(method='bfill')
    df['ta_mean_6h'] = df.groupby('branch_id')['ta'].transform(lambda x: x.rolling(6, min_periods=1).mean()).fillna(method='bfill')
    df['ta_mean_12h'] = df.groupby('branch_id')['ta'].transform(lambda x: x.rolling(12, min_periods=1).mean()).fillna(method='bfill')

    df['ta_diff_1h'] = df.groupby('branch_id')['ta'].diff(1).fillna(0)
    df['ta_diff_3h'] = df.groupby('branch_id')['ta'].diff(3).fillna(0)
    df['ws_diff_1h'] = df.groupby('branch_id')['ws'].diff(1).fillna(0)
    df['ws_diff_3h'] = df.groupby('branch_id')['ws'].diff(3).fillna(0)

    df['day_of_year'] = df['tm'].dt.dayofyear
    df['seasonal_center'] = abs(df['day_of_year'] - 183) / 183
    df['season_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
    df['season_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)

    df['hdh_18'] = np.maximum(18 - df['ta'], 0)
    df['hdh_18_24h'] = df.groupby('branch_id')['hdh_18'].transform(lambda x: x.rolling(24, min_periods=1).sum())
    df['hdh_18_72h'] = df.groupby('branch_id')['hdh_18'].transform(lambda x: x.rolling(72, min_periods=1).sum())

    df['ta_drop_6h'] = df.groupby('branch_id')['ta'].diff(6).fillna(0)
    df['ta_drop_12h'] = df.groupby('branch_id')['ta'].diff(12).fillna(0)
    df['ta_drop_6h_only'] = df['ta_drop_6h'].apply(lambda x: x if x < 0 else 0)
    df['ta_drop_12h_only'] = df['ta_drop_12h'].apply(lambda x: x if x < 0 else 0)

    df['windchill'] = 13.12 + 0.6215*df['ta'] - 11.37*(df['ws']**0.16) + 0.3965*df['ta']*(df['ws']**0.16)
    df['extreme_cold_0'] = (df['windchill'] <= 0).astype(int)
    df['extreme_cold_m10'] = (df['windchill'] <= -10).astype(int)

    df['is_summer'] = df['month'].isin([6,7,8]).astype(int)
    df['low_demand_summer'] = ((df['is_summer'] == 1) & (df['ta'] > 22)).astype(int)

    df['mean_ta_3d'] = df.groupby('branch_id')['ta'].transform(lambda x: x.rolling(24*3, min_periods=1).mean())
    df['cold_wave_flag'] = (df['mean_ta_3d'] <= 0).astype(int)
    df['heat_wave_flag'] = (df['mean_ta_3d'] >= 28).astype(int)

    # 불필요한 컬럼 자동 제거
    for col in ['day', 'hour', 'ws_stl_trend', 'hm']:
        if col in df.columns:
            df = df.drop(columns=[col])

    return df

# 4. 전처리 함수 적용
df_train = preprocess_weather_data(df_train)
df_test = preprocess_weather_data(df_test)

# 5. 범주형 변수 지정
cat_cols = [
    'peak_time1', 'peak_time2', 'heating_season', 'month',
    'temp_category16', 'temp_category18', 'temp_category20'
]
exclude_cols = ['heat_demand'] + cat_cols

num_cols = [col for col in df_train.select_dtypes(include=[np.number]).columns
            if col not in exclude_cols]

for col in cat_cols:
    if col in df_train.columns:
        df_train[col] = df_train[col].astype('category')
    if col in df_test.columns:
        df_test[col] = df_test[col].astype('category')

# 6. OneHotEncoder
ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
ohe.fit(df_train[cat_cols])

train_cat = ohe.transform(df_train[cat_cols])
test_cat = ohe.transform(df_test[cat_cols])

ohe_columns = ohe.get_feature_names_out(cat_cols)

df_train_cat = pd.DataFrame(train_cat, columns=ohe_columns, index=df_train.index)
df_test_cat = pd.DataFrame(test_cat, columns=ohe_columns, index=df_test.index)

# 7. StandardScaler
scaler = StandardScaler()
scaler.fit(df_train[num_cols])

df_train_num = pd.DataFrame(scaler.transform(df_train[num_cols]), columns=num_cols, index=df_train.index)
df_test_num = pd.DataFrame(scaler.transform(df_test[num_cols]), columns=num_cols, index=df_test.index)

# 8. 나머지 컬럼 (타겟 등)
other_cols = [col for col in df_train.columns if col not in num_cols + cat_cols]
df_train_other = df_train[other_cols].copy()
df_test_other = df_test[other_cols].copy()

# 9. 인덱스 정합성 및 정렬 (reset_index로 인덱스 일치 보장)
df_train_num = df_train_num.reset_index(drop=True)
df_train_cat = df_train_cat.reset_index(drop=True)
df_train_other = df_train_other.reset_index(drop=True)
df_test_num = df_test_num.reset_index(drop=True)
df_test_cat = df_test_cat.reset_index(drop=True)
df_test_other = df_test_other.reset_index(drop=True)

assert len(df_train_num) == len(df_train_cat) == len(df_train_other), "Train 데이터 행 개수 불일치"
assert len(df_test_num) == len(df_test_cat) == len(df_test_other), "Test 데이터 행 개수 불일치"

# 10. 최종 병합
df_train_final = pd.concat([df_train_num, df_train_cat, df_train_other], axis=1)
df_test_final = pd.concat([df_test_num, df_test_cat, df_test_other], axis=1)

# 11. heat_demand 결측치 체크 및 경고
n_nan = df_train_final['heat_demand'].isna().sum()
if n_nan > 0:
    print(f"⚠️ [경고] heat_demand 결측치 {n_nan}개 존재. 필요시 결측치 처리 후 저장하세요.")

# 12. 저장
df_train_final.to_csv('df_train_prescale.csv', index=False)
df_test_final.to_csv('df_test_prescale.csv', index=False)


In [10]:
df_train = pd.read_csv('df_train_prescale.csv')
df_test = pd.read_csv('df_test_prescale.csv')
df_train = df_train.sort_values(['branch_id', 'tm'])
df_test = df_test.sort_values(['branch_id', 'tm'])




df_train = df_train.drop(columns=['year'])
df_test = df_test.drop(columns=['year'])
df_train = df_train.set_index('tm')
df_test = df_test.set_index('tm')
df_train = df_train.sort_index()
df_test = df_test.sort_index()

In [11]:
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import pandas as pd
import xgboost as xgb
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance

# ----------------------------
# 1. 학습 함수 정의 (학습은 log1p, 평가는 원래 스케일)
# ----------------------------
def run_6fold_pipeline_with_xgb_timeseries(df_train, df_test, target_col='heat_demand'):
    features = [col for col in df_train.columns if col != target_col]

    # ⭐ 원래 스케일 target 따로 저장
    y_orig = df_train[target_col].copy()
    X = df_train[features]
    y_log = np.log1p(y_orig)

    tscv = TimeSeriesSplit(n_splits=6)
    val_rmses = []
    test_preds = []
    xgb_models = []

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train_log, y_val_log = y_log.iloc[train_idx], y_log.iloc[val_idx]
        y_train_orig, y_val_orig = y_orig.iloc[train_idx], y_orig.iloc[val_idx]  # ✅ 원래 스케일 유지

        print(f"===== Fold {fold+1} =====")

        def xgb_objective(trial):
            params = {
                'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'booster': 'gbtree',
                'tree_method': 'hist',
                'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.2, log=True),
                'max_depth': trial.suggest_int('max_depth', 3, 16),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                'subsample': trial.suggest_float('subsample', 0.7, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
                'gamma': trial.suggest_float('gamma', 0, 5),
                'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
                'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
                'n_estimators': 1000,
                'random_state': 42,
                'n_jobs': -1
            }
            model = xgb.XGBRegressor(**params)
            model.fit(X_train, y_train_log, eval_set=[(X_val, y_val_log)], verbose=False)
            val_pred_log = model.predict(X_val)
            val_pred_orig = np.expm1(val_pred_log)
            return np.sqrt(mean_squared_error(y_val_orig, val_pred_orig))  # ✅ 진짜 원래 스케일로 평가

        study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
        study.optimize(xgb_objective, n_trials=50, show_progress_bar=True)

        best_params = study.best_params
        best_params.update({
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'booster': 'gbtree',
            'tree_method': 'hist',
            'n_estimators': 1000,
            'random_state': 42,
            'n_jobs': -1
        })

        model = xgb.XGBRegressor(**best_params)
        model.fit(X_train, y_train_log, eval_set=[(X_val, y_val_log)], verbose=False)

        val_pred_log = model.predict(X_val)
        val_pred_orig = np.expm1(val_pred_log)
        test_pred_log = model.predict(df_test[features])
        test_pred_orig = np.expm1(test_pred_log)

        val_rmse = np.sqrt(mean_squared_error(y_val_orig, val_pred_orig))
        print(f"Fold {fold+1} RMSE (원래 스케일): {val_rmse:.4f}")

        val_rmses.append(val_rmse)
        test_preds.append(test_pred_orig)
        xgb_models.append(model)

    avg_val_rmse = np.mean(val_rmses)
    avg_test_pred = np.mean(test_preds, axis=0)

    return {
        'val_rmse': avg_val_rmse,
        'val_rmses': val_rmses,
        'test_pred': avg_test_pred,
        'test_index': df_test.index,
        'xgb_models': xgb_models,
        'features': features
    }

In [12]:
branch_rmse_results = {}
test_pred_dict = {}

branch_ids = df_train['branch_id'].unique()

for branch in branch_ids:
    train_branch = df_train[df_train['branch_id'] == branch].copy()
    test_branch = df_test[df_test['branch_id'] == branch].copy()
    
    # branch_id는 모델에 불필요하면 제거
    train_branch = train_branch.drop(columns=['branch_id'])
    test_branch = test_branch.drop(columns=['branch_id'])
    
    target_col = 'heat_demand'
    
    results = run_6fold_pipeline_with_xgb_timeseries(train_branch, test_branch, target_col)
    
    branch_rmse_results[branch] = {
        'val_rmse': results['val_rmse']
    }
    
    test_pred_dict[branch] = pd.DataFrame({
        'branch_ID': branch,
        'TM': results['test_index'],
        'heat_demand': results['test_pred']
    }).set_index('TM')


# test_pred_dict의 모든 branch 결과를 하나의 DataFrame으로 병합
merged_df = pd.concat(test_pred_dict.values()).reset_index()


# CSV 파일로 저장
merged_df.to_csv('250464_test.csv', index=False)  


[I 2025-06-27 02:28:04,874] A new study created in memory with name: no-name-0ca7cda6-c614-4172-8a4a-b5d494cf10a4


===== Fold 1 =====


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-06-27 02:28:06,338] Trial 0 finished with value: 18.504939673430105 and parameters: {'learning_rate': 0.00727491708802781, 'max_depth': 16, 'min_child_weight': 8, 'subsample': 0.8795975452591109, 'colsample_bytree': 0.7468055921327309, 'gamma': 0.7799726016810132, 'reg_lambda': 0.5808361216819946, 'reg_alpha': 8.661761457749352}. Best is trial 0 with value: 18.504939673430105.
[I 2025-06-27 02:28:07,521] Trial 1 finished with value: 15.029543546546863 and parameters: {'learning_rate': 0.02416482602989751, 'max_depth': 12, 'min_child_weight': 1, 'subsample': 0.9909729556485982, 'colsample_bytree': 0.9497327922401265, 'gamma': 1.0616955533913808, 'reg_lambda': 1.8182496720710062, 'reg_alpha': 1.8340450985343382}. Best is trial 1 with value: 15.029543546546863.
[I 2025-06-27 02:28:09,155] Trial 2 finished with value: 16.45597790495836 and parameters: {'learning_rate': 0.005012686302434875, 'max_depth': 10, 'min_child_weight': 5, 'subsample': 0.7873687420594125, 'colsample_bytree':

[I 2025-06-27 02:29:03,402] A new study created in memory with name: no-name-d25cc8be-5cc7-4017-a770-e4bc57da9e4d


...

[I 2025-06-27 06:16:40,980] Trial 0 finished with value: 22.96195874977118 and parameters: {'learning_rate': 0.00727491708802781, 'max_depth': 16, 'min_child_weight': 8, 'subsample': 0.8795975452591109, 'colsample_bytree': 0.7468055921327309, 'gamma': 0.7799726016810132, 'reg_lambda': 0.5808361216819946, 'reg_alpha': 8.661761457749352}. Best is trial 0 with value: 22.96195874977118.
[I 2025-06-27 06:16:42,729] Trial 1 finished with value: 23.091884991885884 and parameters: {'learning_rate': 0.02416482602989751, 'max_depth': 12, 'min_child_weight': 1, 'subsample': 0.9909729556485982, 'colsample_bytree': 0.9497327922401265, 'gamma': 1.0616955533913808, 'reg_lambda': 1.8182496720710062, 'reg_alpha': 1.8340450985343382}. Best is trial 0 with value: 22.96195874977118.
[I 2025-06-27 06:16:45,947] Trial 2 finished with value: 22.74804620139986 and parameters: {'learning_rate': 0.005012686302434875, 'max_depth': 10, 'min_child_weight': 5, 'subsample': 0.7873687420594125, 'colsample_bytree': 0.

[I 2025-06-27 06:18:48,669] A new study created in memory with name: no-name-542c12d7-7655-4c4a-be52-ac07218dd893


Fold 5 RMSE (원래 스케일): 20.5082
===== Fold 6 =====


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-06-27 06:18:51,467] Trial 0 finished with value: 25.531120197711708 and parameters: {'learning_rate': 0.00727491708802781, 'max_depth': 16, 'min_child_weight': 8, 'subsample': 0.8795975452591109, 'colsample_bytree': 0.7468055921327309, 'gamma': 0.7799726016810132, 'reg_lambda': 0.5808361216819946, 'reg_alpha': 8.661761457749352}. Best is trial 0 with value: 25.531120197711708.
[I 2025-06-27 06:18:53,400] Trial 1 finished with value: 25.439078194162036 and parameters: {'learning_rate': 0.02416482602989751, 'max_depth': 12, 'min_child_weight': 1, 'subsample': 0.9909729556485982, 'colsample_bytree': 0.9497327922401265, 'gamma': 1.0616955533913808, 'reg_lambda': 1.8182496720710062, 'reg_alpha': 1.8340450985343382}. Best is trial 1 with value: 25.439078194162036.
[I 2025-06-27 06:18:56,933] Trial 2 finished with value: 25.39994796959116 and parameters: {'learning_rate': 0.005012686302434875, 'max_depth': 10, 'min_child_weight': 5, 'subsample': 0.7873687420594125, 'colsample_bytree':

In [13]:
df1 = pd.read_csv('250464_test.csv')
df1['TM'] = pd.to_datetime(df1['TM'])
df1 = df1.sort_values(by=['branch_ID', 'TM']).reset_index(drop=True)
df2 = pd.read_csv('test_heat.csv')
df2['heat_demand'] = df1['heat_demand'].round(1)
df2.to_csv('250464.csv', index=False) 